<a href="https://colab.research.google.com/github/harshithgowdakc/NYC-Taxi-Data/blob/main/NYC_Taxi_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from haversine import haversine
import statsmodels.formula.api as sm
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import warnings; warnings.simplefilter('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AlmaBetter/Project CPG/NYC Taxi Data.csv')

In [5]:
data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [6]:
data.shape 

(1458644, 11)

In [7]:
#Check count of unique id's in the dataset
print("There are %d unique id's in Training dataset, which is equal to the number of records"%(data.id.nunique()))

There are 1458644 unique id's in Training dataset, which is equal to the number of records


In [8]:
#Check for NaN values
data.isnull().sum()

id                    0
vendor_id             0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dtype: int64

In [9]:
#Convert timestamp to datetime format to fetch the other details as listed below
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])
data['dropoff_datetime'] = pd.to_datetime(data['dropoff_datetime'])

In [10]:
#Calculate and assign new columns to the dataframe such as weekday,
#month and pickup_hour which will help us to gain more insights from the data.
data['weekday'] = data.pickup_datetime.dt.weekday
data['month'] = data.pickup_datetime.dt.month
data['weekday_num'] = data.pickup_datetime.dt.weekday
data['pickup_hour'] = data.pickup_datetime.dt.hour

In [11]:
def calc_distance(df):
    pickup = (df['pickup_latitude'], df['pickup_longitude'])
    drop = (df['dropoff_latitude'], df['dropoff_longitude'])
    return haversine(pickup, drop)

In [ ]:
#Calculate distance and assign new column to the dataframe.
data['distance'] = data.apply(lambda x: calc_distance(x), axis = 1)

In [ ]:
#Calculate Speed in km/h for further insights
data['speed'] = (data.distance/(data.trip_duration/3600))

In [ ]:
#Check the type of each variable
data.dtypes.reset_index()

In [ ]:
#Dummify all the categorical features like "store_and_fwd_flag, vendor_id, month, weekday_num, pickup_hour, passenger_count" except the label i.e. "trip_duration"

dummy = pd.get_dummies(data.store_and_fwd_flag, prefix='flag')
dummy.drop(dummy.columns[0], axis=1, inplace=True) #avoid dummy trap
data = pd.concat([data,dummy], axis = 1)

dummy = pd.get_dummies(data.vendor_id, prefix='vendor_id')
dummy.drop(dummy.columns[0], axis=1, inplace=True) #avoid dummy trap
data = pd.concat([data,dummy], axis = 1)

dummy = pd.get_dummies(data.month, prefix='month')
dummy.drop(dummy.columns[0], axis=1, inplace=True) #avoid dummy trap
data = pd.concat([data,dummy], axis = 1)

dummy = pd.get_dummies(data.weekday_num, prefix='weekday_num')
dummy.drop(dummy.columns[0], axis=1, inplace=True) #avoid dummy trap
data = pd.concat([data,dummy], axis = 1)

dummy = pd.get_dummies(data.pickup_hour, prefix='pickup_hour')
dummy.drop(dummy.columns[0], axis=1, inplace=True) #avoid dummy trap
data = pd.concat([data,dummy], axis = 1)

dummy = pd.get_dummies(data.passenger_count, prefix='passenger_count')
dummy.drop(dummy.columns[0], axis=1, inplace=True) #avoid dummy trap
data = pd.concat([data,dummy], axis = 1)

In [ ]:
data.head()

In [ ]:
pd.options.display.float_format = '{:.2f}'.format #To suppres scientific notation.
data.passenger_count.value_counts()

In [ ]:
plt.figure(figsize = (20,5))
sns.boxplot(data.passenger_count)
plt.show()

In [ ]:
data.passenger_count.describe()

In [ ]:
data['passenger_count'] = data.passenger_count.map(lambda x: 1 if x == 0 else x)

In [ ]:
data = data[data.passenger_count <= 6]

In [ ]:
data.passenger_count.value_counts()


In [ ]:
sns.countplot(data.passenger_count)
plt.show()

### Vendor

Here we analyze taxi data only for the 2 vendors which are listed as 1 and 2 in the datset.

In [ ]:
sns.countplot(data.vendor_id)
plt.show()

Though both the vendors seems to have almost equal market share. But Vendor 2 is evidently more famous among the population as per the above graph.

### Distance

Let's now have a look on the distribution of the distance across the different types of rides.


In [ ]:
print(data.distance.describe())

In [ ]:
plt.figure(figsize = (20,5))
sns.boxplot(data.distance)
plt.show()

**Interesting find:**
There some trips with over 100 km distance.

Some of the trips distance value is 0 km.

**Observations:**

mean distance travelled is approx 3.5 kms.

standard deviation of 4.3 which shows that most of the trips are limited to the range of 1-10 kms.

In [ ]:
print("There are {} trip records with 0 km distance".format(data.distance[data.distance == 0 ].count()))

In [ ]:
data[data.distance == 0 ].head()

**Observations**

Around 6K trip record with distance equal to 0. Below are some possible explanation for such records.



1.   Customer changed mind and cancelled the journey just after accepting it.
2.   Software didn't recorded dropoff location properly due to which dropoff location is the same as the pickup location.
3.   Issue with GPS tracker while the journey is being finished.
4.   Driver cancelled the trip just after accepting it due to some reason. So the trip couldn't start
5.   Or some other issue with the software itself which a technical guy can explain

There is some serious inconsistencies in the data where drop off location is same as the pickup location. We can't think off imputing the distance values considering a correlation with the duration because the dropoff_location coordinates would not be inline with the distance otherwise. We will look more to it in bivariate analysis with the Trip duration.

In [ ]:
data.distance.groupby(pd.cut(data.distance, np.arange(0,100,10))).count().plot(kind='barh')
plt.show()

From the above observation it is evident that most of the rides are completed between 1-10 Kms with some of the rides with distances between 10-30 kms. Other slabs bar are not visible because the number of trips are very less as compared to these slabs

### Trip duration

In [ ]:
data.trip_duration.describe()

In [ ]:
plt.figure(figsize = (20,5))
sns.boxplot(data.trip_duration)
plt.show()

Interesting find:
Some trip durations are over 100000 seconds which are clear outliers and should 
*   Some trip durations are over 100000 seconds which are clear outliers and should be removed.

Observations:

*   There are some durations with as low as 1 second. which points towards trips with 0 km distance.
*   Major trip durations took between 10-20 mins to complete.
*   Mean and mode are not same which shows that trip duration distribution is skewed towards right.

Let's analyze more

In [ ]:
data.trip_duration.groupby(pd.cut(data.trip_duration, np.arange(1,max(data.trip_duration),3600))).count()

Observations:

*   There are some trips with more than 24 hours of travel duration i.e. 86400 seconds. Which might have occured on weekends for the outstation travels.
*   Major chunk of trips are completed within an interval of 1 hour with some good numbers of trips duration going above 1 hour.

Let's look at those trips with huge duration, these are outliers and should be removed for the data consistency.

In [ ]:
data[data.trip_duration > 86400]

Observations:

*   These trips ran for more than 20 days, which seems unlikely by the distance travelled.
*   All the trips are taken by vendor 1 which points us to the fact that this vendor might allows much longer trip for outstations.
*   All these trips are either taken on Tuesday's in 1st month or Saturday's in 2nd month. There might be some relation with the weekday, pickup location, month and the passenger.
*   But they fail our purpose of correct prediction and bring inconsistencies in the algorithm calculation.

We should get rid of them for the sake of data consistency. Those are black swans !!

In [ ]:
data = data[data.trip_duration <= 86400]

Let's visualize the number of trips taken in slabs of 0-10, 20-30 ... minutes respectively

In [ ]:
data.trip_duration.groupby(pd.cut(data.trip_duration, np.arange(1,7200,600))).count().plot(kind='barh')
plt.xlabel('Trip Counts')
plt.ylabel('Trip Duration (seconds)')
plt.show()

We can observe that most of the trips took 0 - 30 mins to complete i.e. approx 1800 secs. Let's move ahead to next feature.

### Speed

Speed is a function of distance and time. Let's visualize speed in different trips.

Maximum speed limit in NYC is as follows:

*   25 mph in urban area i.e. 40 km/h

*   65 mph on controlled state highways i.e. approx 104 km/h
 

In [ ]:
data.speed.describe()

In [ ]:
plt.figure(figsize = (20,5))
sns.boxplot(data.speed)
plt.show()

Interesting find:

*   Many trips were done at a speed of over 200 km/h. Going SuperSonic..!!

Let's remove them and focus on the trips which were done at less than 104 km/h as per the speed limits

In [ ]:
data = data[data.speed <= 104]
plt.figure(figsize = (20,5))
sns.boxplot(data.speed)
plt.show()

Observations:


*   Trips over 30 km/h are being considered as outliers but we cannot ignore them because they are well under the highest speed limit of 104 km/h on state controlled highways.
*   Mostly trips are done at a speed range of 10-20 km/h with an average speed of around 14 km/h.

Let's take a look at the speed range ditribution with the help of graph.

In [ ]:
data.speed.groupby(pd.cut(data.speed, np.arange(0,104,10))).count().plot(kind = 'barh')
plt.xlabel('Trip count')
plt.ylabel('Speed (Km/H)')
plt.show()

It is evident from this graph what we thought off earlier i.e. most of the trips were done at a speed range of 10-20 km/H.

### Store_and_fwd_flag

This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip.

In [ ]:
data.flag_Y.value_counts(normalize=True)

Observations:


*   List item


1.   List item

1.   List item
2.   List item


2.   List item


*   List item



In [ ]:
data.flag_Y.value_counts()

Above result shows that around 8K trips had to store the flag and then report to the server when the connection was established. Let's check the respective distribution with the vendors for the offline trips.

In [ ]:
data.vendor_id[data.flag_Y == 1].value_counts()

Observations:

In [ ]:
data[data.flag_Y == 1]

Observations



1.   List item

*   List item
*   List item


2.   List item



### Total trips Per Hour

Let's take a look at the distribution of the pickups across the 24 hour time scale.

In [ ]:
sns.countplot(data.pickup_hour)
plt.show()

Observation

*   It's inline with the general trend of taxi pickups which starts increasing from 6AM in the morning and then declines from late evening i.e. around 8 PM. There is no unusual behavior here.

### Total trips per weekday

Let's take a look now at the distribution of taxi pickups across the week.

In [ ]:
plt.figure(figsize = (8,6))
sns.countplot(data.weekday_num)
plt.xlabel(' Month ')
plt.ylabel('Pickup counts')
plt.show()

Observation
*   Here we can see an increasing trend of taxi pickups starting from Monday till Friday. The trend starts declining from saturday till monday which is normal where some office going people likes to stay at home for rest on the weekends.

Let's drill down more to see the hourwise pickup pattern across the week

In [ ]:
n = sns.FacetGrid(data, col='weekday_num')
n.map(plt.hist, 'pickup_hour')
plt.show()

Interesting find:
*   Taxi pickups increased in the late night hours over the weekend possibly due to more outstation rides or for the late night leisures nearby activities.


*   Early morning pickups i.e before 5 AM have increased over the weekend in comparison to the office hours pickups i.e. after 7 AM which have decreased due to obvious reasons.

*   Taxi pickups seems to be consistent across the week at 15 Hours i.e. at 3 PM.

### Total trips per month

Let's take a look at the trip distribution across the months to understand if there is any diffrence in the taxi pickups in different months

In [ ]:
sns.countplot(data.month)
plt.ylabel('Trip Counts')
plt.xlabel('Months')
plt.show()

Quite a balance across the months here. It could have been more equivalent if we wouldn't have removed the inconsistent records in our study of the univariate analysis

## **Bivariate Analysis**

Bivariate analysis is used to find out if there is a relationship between two sets of values. It usually involves the variables X and Y.

In [ ]:
group1 = data.groupby('pickup_hour').trip_duration.mean()
sns.pointplot(group1.index, group1.values)
plt.ylabel('Trip Duration (seconds)')
plt.xlabel('Pickup Hour')
plt.show()

In [ ]:
group2 = data.groupby('weekday_num').trip_duration.mean()
sns.pointplot(group2.index, group2.values)
plt.ylabel('Trip Duration (seconds)')
plt.xlabel('Weekday')
plt.show()

In [ ]:
group3 = data.groupby('month').trip_duration.mean()
sns.pointplot(group3.index, group3.values)
plt.ylabel('Trip Duration (seconds)')
plt.xlabel('Month')
plt.show()

### Trip duration per vendor

We can also look at the average difference between the trip duration for each vendor. However we do know that vendor 2 has larger share of the market. Let's visualize.

In [ ]:
group4 = data.groupby('vendor_id').trip_duration.mean()
sns.barplot(group4.index, group4.values)
plt.ylabel('Trip Duration (seconds)')
plt.xlabel('Vendor')
plt.show()

Vendor 2 takes the crown. Average trip duration for vendor 2 is higher than vendor 1 by approx 200 seconds i.e. atleast 3 minutes per trip.

### Trip duration v/s Flag

Let's visualize if there is any effect of flag setting on the trip duration?

In [ ]:
plt.figure(figsize = (6,5))
plot_dur = data.loc[(data.trip_duration < 10000)]
sns.boxplot(x = "flag_Y", y = "trip_duration", data = plot_dur)
plt.show()

Observations:

*   Trip durations scale is less for the trips where the flag is set i.e. the trip details are stored before sending it to the server.
*   Trip duration outliers are also less for the trips with flag 'Y' as compared the trips with flag 'N'.
*   Trip duration is longer for the trips where the flag is not set.
*   Inter quartile range of trip duration is more for the trips with the flag 'Y' as compared to the trips with flag 'N' but the median value is almost equal for both.

### Distance per hour

Now, let us check how the distance is distributed against different variables. We know that trip distance must be more or less proportional to the trip duration if we ignore general traffic and other stuff on the road. Let's visualize this for each hour now.

Since we have already done the outlier analysis for this variable as well. We can take the mean as aggregate measure for our visualizations.

In [ ]:
group5 = data.groupby('pickup_hour').distance.mean()
sns.pointplot(group5.index, group5.values)
plt.ylabel('Distance (km)')
plt.show()

Observations:


*   Trip distance is highest during early morning hours which can account for some things like:
    1.   Outstation trips taken during the weekends.

    2.   Longer trips towards the city airport which is located in the outskirts of the city.



*   Trip distance is fairly equal from morning till the evening varying around 3 - 3.5 kms.

*   It starts increasing gradually towards the late night hours starting from evening till 5 AM and decrease steeply towards morning.

In [ ]:
pd.set_option('display.max_columns', None)

### Distance per weekday

Let's analyze the average trip distance covered on each day of the week.

In [ ]:
group6 = data.groupby('weekday_num').distance.mean()
sns.pointplot(group6.index, group6.values)
plt.ylabel('Distance (km)')
plt.show()

So it's a fairly equal distribution with average distance metric verying around 3.5 km/h with Sunday being at the top may be due to outstation trips or night trips towards the airport.

### Distance per month

Now we will look at the average trip distance covered per month.

In [ ]:
group6 = data.groupby('month').distance.mean()
sns.pointplot(group6.index, group6.values)
plt.ylabel('Distance (km)')
plt.show()

Here also the distibution is almost equivalent, varying mostly around 3.5 km/h with 5th month being the highest in the average distance and 2nd month being the lowest.

Distance per vendor

In [ ]:
group8 = data.groupby('vendor_id').distance.mean()
sns.barplot(group8.index, group8.values)
plt.ylabel("Distance km")
plt.show()

This is more or less same picture with both the vendors. Nothing more to analyze in this.

### Distance v/s Flag

Let's visualize if there is any effect of Flag setting on the distance covered in the trips

In [ ]:
plt.figure(figsize = (6,6))
plot_dist = data.loc[(data.distance < 100)]
sns.boxplot(x = "flag_Y", y = "distance", data = plot_dist)
plt.ylabel('Distance (km)')
plt.show()

Observations:


*   We can see almost similar results like the one observed in the Trip duration v/s Flag analysis.

*   Only two major difference can be seen here.
    1.   Interquartile range of distance is almost twice for Flag 'Y' trips as compared to the Flag 'N' trips

    2.   Median value is much different in both the case as well.

Which points us to the fact that range of distance and trip duration for the Flag 'Y' trips is much more limited and confined as compared with the flag 'N' trips and this also resulted in much less number of outliers for Flag 'Y' trips.

### Distance v/s Trip duration

Let's visualize the relationship between Distance covered and respective trip duration.

In [ ]:
plt.scatter(data.trip_duration, data.distance , s=1, alpha=0.5)
plt.ylabel('Distance')
plt.xlabel('Trip Duration')
plt.show()

Interesting find:


*   There are lots of trips which covered negligible distance but clocked more than 20,000 seconds in terms of the Duration.

*   Initially there is some proper correlation between the distance covered and the trip duration in the graph. but later on it all seems uncorrelated.

*   There were few trips which covered huge distance of approx 200 kms within very less time frame, which is unlikely and should be treated as outliers.


Let's focus on the graph area where distance is < 50 km and duration is < 1000 seconds.


In [ ]:
dur_dist = data.loc[(data.distance < 50) & (data.trip_duration < 1000)]
plt.scatter(dur_dist.trip_duration, dur_dist.distance , s=1, alpha=0.5)
plt.ylabel('Distance')
plt.xlabel('Trip Duration')
plt.show()

Observations:

*   There should have been a linear relationship between the distance covered and trip duration on an average but we can see dense collection of the trips in the lower right corner which showcase many trips with the inconsistent readings.

Idea:
We should remove those trips which covered 0 km distance but clocked more than 1 minute to make our data more consistent for predictive model. Because if the trip was cancelled after booking, than that should not have taken more than a minute time. This is our assumption.

In [ ]:
data = data[~((data.distance == 0) & (data.trip_duration >= 60))]

Now, Instead of looking at each and every trip, we should approximate and try to filter those trips which covered less than 1 km distance and but clocked more than an hour.

In [ ]:
duo = data.loc[(data['distance'] <= 1) & (data['trip_duration'] >= 3600),['distance','trip_duration']].reset_index(drop=True)

In [ ]:
duo.head(2)

In [ ]:
sns.regplot(duo.distance, duo.trip_duration)
plt.show()

Observations:


*   Though the straight line tries to show some linear relation between the two. But there seems to be negligible correlation between these two metric as seen from the scatter plot where it should have been a linear distribution.

*   It is rarely occurs that customer keep sitting in the taxi for more than an hour and it does not travel for even 1 km.

These should be removed to bring in more consistency to our results.

In [ ]:
data = data[~((data['distance'] <= 1) & (data['trip_duration'] >= 3600))]

In [ ]:
sns.regplot(data.distance, data.trip_duration)
plt.show()

### Average speed per hour

Let's look at the average speed of NYC Taxi per hour.

In [ ]:
group9 = data.groupby('pickup_hour').speed.mean()
sns.pointplot(group9.index, group9.values)
plt.show()

Observation:

*   The average trend is totally inline with the normal circumstances.
*   Average speed tend to increase after late evening and continues to increase gradually till the late early morning hours.
*   Average taxi speed is highest at 5 AM in the morning, then it declines steeply as the office hours approaches.
*   Average taxi speed is more or less same during the office hours i.e. from 8 AM till 6PM in the evening.

### Average speed per weekday

Let's visualize that on an average what is the speed of a taxi on any given weekday.

In [ ]:
group10 = data.groupby('weekday_num').speed.mean()
sns.pointplot(group10.index, group10.values)
plt.show()

Observations:



*   Average taxi speed is higher on weekend as compared to the weekdays which is obvious when there is mostly rush of office goers and business owners.

*   Even on monday the average taxi speed is shown higher which is quite surprising when it is one of the most busiest day after the weekend. There can be several possibility for such behaviour

    1.   Lot of customers who come back from outstation in early hours of Monday before 6 AM to attend office on time.
    2.   Early morning hours customers who come from the airports after vacation to attend office/business on time for the coming week.

*   There could be some more reasons as well which only a local must be aware of.
*  We also can't deny the anomalies in the dataset. which is quite cumbersome to spot in such a large dataset.

### Passenger count per vendor

Let's try some different metric in the series i.e. passenger count. We will plot it agaist the vendor only because it will not be much helpful to plot it against hour, weekday or month like others as the passenger count should be a whole number and not a ratio.

we will take mean as the aggregate measure because we already did the outlier analysis on this metric. So our results woudn't be affected by some extreme values. Also if we take median than it will return only 1 because majorty of the trips have been taken by single passenger. Let's take a look about it's distribution.

In [ ]:
group9 = data.groupby('vendor_id').passenger_count.mean()
sns.barplot(group9.index, group9.values)
plt.ylabel('Passenger count')
plt.show()

Clear difference between the two operators for the average passenger count in all trips. It seems that vendor 2 trips generally consist of 2 passengers as compared to the vendor 1 with 1 passenger. Let's bifurcate it further.

In [ ]:
data.groupby('passenger_count').vendor_id.value_counts().reset_index(name='count').pivot("passenger_count","vendor_id","count").plot(kind='bar')
plt.show()

Interesting find:


*   It seems that most of the big cars are served by the Vendor 2 including minivans because other than passenger 1, vendor 2 has majority in serving more than 1 passenger count and that explains it greater share of the market.

# Feature Engineering

After looking at the dataset from different perspectives. Let's prepare our dataset before training our model. Since our dataset do not contain very large number of dimensions. We will first try to use feature selection instead of the feature extraction technique.

Question:
But what's the difference?

Feature selection: we select a subset of the original feature set based on the statistical significance of different parameters.
Example: Backward elimination, Forward selection, Recursive feature elimination

Feature extraction: we build a new set of features from the original feature set
Example: PCA, LDA, Kernel PCA

## Feature Selection

Intuition:

*   We will use backward elimination technique to select the best features to train our model.

*   It displays some statistical metrics with there significance value.

*   Like, It shows the p values for each feature as per its significance in the whole dataset.

*   It also shows the adjusted R squared values to identify whether removing or selecting the feature is beneficial or not.

*   For now we will only look at the P and adjusted R squared value to decide which features to keep and which needed to be removed.

Let's assign the values to X & Y array from the dataset.

In [ ]:
#First chech the index of the features and label
list(zip( range(0,len(data.columns)),data.columns))

In [ ]:
Y = data.iloc[:,10].values
X = data.iloc[:,range(15,61)].values

Question:
Why few features are not assigned to the X array like features at the index 2,3,10 were missed?

Idea:

*   duration variable assigned to Y because that is the dependent variable.

*   features such as id, timestamp and weekday were not assigned to X array because they are of type object. And we need an array of float data type.

Trick for backward elimination:
General equation for multiple linear regression is like

Y = a0 + a1x1 + a2x2 + ... + anxn

Since, we dont have x0 in our X array so the regressor won't consider the constant value of the equation i.e. a0. So to make it count in the equation we will append the selected feature set with a contant series of 1's as a first column. To make it appear like below equation to the statsmodel.

y = a0x0 + a1x1 + a2x2 + ... + anxn

In [ ]:
print("Let's append {} rows of 1's as the first column in the X array".format(X.shape[0]))

In [ ]:
X1 = np.append(arr = np.ones((X.shape[0],1)).astype(int), values = X, axis = 1)

In [ ]:
X1.shape

There we go, our feature set is now ready for the feature selection model with 1s in the first column for a0 constant.

Let's fit stats model on the X array to figure out an optimal set of features by recursively checking for the highest p value and removing the feature of that index.

Note:
Here we will take the level of significance as 0.05 i.e. 5% which means that we will reject feature from the list of array and re-run the model till p value for all the features goes below .05 to find out the optimal combination for our model.

In [ ]:
#Select all the features in X array
X_opt = X1[:,range(0,46)]
regressor_OLS = sm.OLS(endog = Y, exog = X_opt).fit()

#Fetch p values for each feature
p_Vals = regressor_OLS.pvalues

#define significance level for accepting the feature.
sig_Level = 0.05

#Loop to iterate over features and remove the feature with p value less than the sig_level
while max(p_Vals) > sig_Level:
    print("Probability values of each feature \n")
    print(p_Vals)
    X_opt = np.delete(X_opt, np.argmax(p_Vals), axis = 1)
    print("\n")
    print("Feature at index {} is removed \n".format(str(np.argmax(p_Vals))))
    print(str(X_opt.shape[1]-1) + " dimensions remaining now... \n")
    regressor_OLS = sm.OLS(endog = Y, exog = X_opt).fit()
    p_Vals = regressor_OLS.pvalues
    print("=================================================================\n")
    
#Print final summary
print("Final stat summary with optimal {} features".format(str(X_opt.shape[1]-1)))
regressor_OLS.summary()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, random_state=4, test_size=0.2)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_pca = scaler.fit_transform(X_train_pca)
X_test_pca = scaler.transform(X_test_pca)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA().fit(X_train_pca)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel("number of components")
plt.ylabel("Cumulative explained variance")
plt.show()

In [ ]:
#Linear regressor for the raw data
regressor = LinearRegression() 
regressor.fit(X_train,y_train) 


In [ ]:
#Predict from the test features of raw data
y_pred = regressor.predict(X_test) 

In [ ]:
#Evaluate the regressor on the raw data
print('RMSE score for the Multiple LR raw is : {}'.format(np.sqrt(metrics.mean_squared_error(y_test,y_pred))))
print('Variance score for the Multiple LR raw is : %.2f' % regressor.score(X_test, y_test))
print("\n")

Random Forest Regressor


In [ ]:
#instantiate the object for the Random Forest Regressor with default params from raw data
regressor_rfraw = RandomForestRegressor(n_jobs=-1)

#instantiate the object for the Random Forest Regressor with default params for Feature Selection Group
regressor_rf = RandomForestRegressor(n_jobs=-1)

# #instantiate the object for the Random Forest Regressor with tuned hyper parameters for Feature Selection Group
# regressor_rf1 = RandomForestRegressor(n_estimators = 26,
#                                      max_depth = 22,
#                                      min_samples_split = 9,
#                                      n_jobs=-1)

#instantiate the object for the Random Forest Regressor for Feature Extraction Group
regressor_rf2 = RandomForestRegressor(n_jobs=-1)


#Train the object with default params for raw data
regressor_rfraw.fit(X_train,y_train)

In [ ]:
#Predict the output with object of default params for Feature Selection Group
y_pred_rfraw = regressor_rfraw.predict(X_test)

In [ ]:
#Evaluate the model with default params for raw data
print('RMSE score for the RF regressor raw is : {}'.format(np.sqrt(metrics.mean_squared_error(y_test,y_pred_rfraw))))
print('RMSLE score for the RF regressor raw is : {}'.format(np.sqrt(metrics.mean_squared_log_error(y_test,y_pred_rfraw))))
print('Variance score for the RF regressor raw is : %.2f' % regressor_rfraw.score(X_test, y_test))